1. Reconnect to Milvus

2. Recreate the same embedder (for query embedding only)

3. Point LangChain’s Milvus wrapper at the existing collection

4. Use that as your retriever

In [1]:
from pymilvus import connections
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

# Set-Up

In [2]:
# Reconnect
connections.connect(host="localhost", port="19530")

# Reuse embedding model
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Connect to existing collection
vector_store = Milvus(
    embedding_function=embedder,
    collection_name="medical_knowledge_base_v2",
    connection_args={"host": "localhost", "port": "19530"}
)

# Build retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_72043/2631204849.py:8: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vector_store = Milvus(


In [3]:
import logging
import numpy as np
from typing import List, Any, Dict, Optional, Tuple

from pymilvus import connections
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.agents import AgentExecutor, create_react_agent, AgentType, initialize_agent
from langchain.tools import Tool


# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Open AI Key
from dotenv import load_dotenv
from pathlib import Path
env_path = Path('/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment Research 1 - Agents/.env')
load_dotenv(dotenv_path=env_path)
# print(os.getenv("OPENAI_API_KEY"))


True

# RL

In [4]:
class RLFeedbackSystem:
    def __init__(self, rag_system):
        self.rag_system = rag_system
        self.feedback_history = []
        self.exploration_rate = 0.2
        self.min_k = 3
        self.max_k = 10

    def record_feedback(self, query: str, retrieved_docs: List[Any], answer: str, feedback_score: float):
        self.feedback_history.append({
            "query": query,
            "docs": retrieved_docs,
            "answer": answer,
            "score": feedback_score,
            "params": self.rag_system.retrieval_params.copy()
        })

        if len(self.feedback_history) % 10 == 0:
            self._update_parameters()

    def _update_parameters(self):
        if len(self.feedback_history) < 10:
            return

        recent = self.feedback_history[-10:]
        avg = sum(item["score"] for item in recent) / 10
        current_k = self.rag_system.retrieval_params.get("k", 5)

        if np.random.random() < self.exploration_rate:
            new_k = np.random.randint(self.min_k, self.max_k + 1)
        elif avg < 0.6:
            new_k = min(current_k + 1, self.max_k)
        elif avg > 0.8:
            new_k = max(current_k - 1, self.min_k)
        else:
            new_k = current_k

        self.rag_system.update_retrieval_params({"k": new_k})


# Agent

In [5]:
class AgentSystem:
    def __init__(self, rag_system):
        self.rag_system = rag_system

        self.tools = [
            Tool(
                name="SearchDocs",
                description="Retrieve documents from the knowledge base",
                func=self._search_docs
            ),
            Tool(
                name="SynthesizeAnswer",
                description="Generate a final answer from retrieved documents",
                func=self._synthesize_answer
            )
        ]

        prompt_template = PromptTemplate(
            input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
            template="""You are an intelligent agent using tools to answer questions.

Tools:
{tools}

Use this format:
Question: {input}
Thought: I need to think about how to solve this
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

{agent_scratchpad}"""
)

        self.agent = create_react_agent(
            llm=self.rag_system.llm,
            tools=self.tools,
            prompt=prompt_template
        )
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            max_iterations=10,
            max_execution_time=60.0
        )

    def _search_docs(self, query: str) -> str:
        docs = self.rag_system.retriever.invoke(query)
        return "\n\n".join([d.page_content for d in docs])

    def _synthesize_answer(self, _: str) -> str:
        answer, _ = self.rag_system.answer_question(query)
        return answer

    def process_query(self, query: str) -> Dict[str, Any]:
        self.last_query = query
        return self.agent_executor.invoke({"input": query})


# Vector Store Manager

In [6]:
class VectorStoreManager:
    def __init__(self, embedder, host, port, collection_name):
        connections.connect(host=host, port=port)
        self.vector_store = Milvus(
            embedding_function=embedder,
            collection_name=collection_name,
            connection_args={"host": host, "port": port}
        )

    def get_retriever(self, search_kwargs=None):
        if search_kwargs is None:
            search_kwargs = {"k": 5}
        return self.vector_store.as_retriever(search_kwargs=search_kwargs)


# RAG

In [7]:
class RAGSystem:
    def __init__(self, vector_store_manager, model_name="gpt-4o", temperature=0.1):
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)
        self.vector_store_manager = vector_store_manager
        self.retrieval_params = {"k": 5}
        self.retriever = self.vector_store_manager.get_retriever(self.retrieval_params)

        self.prompt = ChatPromptTemplate.from_template("""
        You are a helpful assistant. Use the context below to answer the question.
        Context:
        {context}

        Question: {question}
        """)
        self.chain = (
            {"context": self.retriever, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

    def answer_question(self, question: str):
        docs = self.retriever.invoke(question)
        answer = self.chain.invoke(question)
        return answer, docs

    def update_retrieval_params(self, params):
        self.retrieval_params.update(params)
        self.retriever = self.vector_store_manager.get_retriever(self.retrieval_params)


# Comprehensive RAG

In [8]:
class ComprehensiveRAGSystem:
    def __init__(
        self,
        embedding_model="sentence-transformers/all-MiniLM-L6-v2",
        llm_model="gpt-4o",
        milvus_host="localhost",
        milvus_port="19530",
        collection_name="medical_knowledge_base_v2"
    ):
        embedder = HuggingFaceEmbeddings(model_name=embedding_model)
        self.vsm = VectorStoreManager(embedder, milvus_host, milvus_port, collection_name)
        self.rag = RAGSystem(self.vsm, model_name=llm_model)
        self.rl = RLFeedbackSystem(self.rag)
        self.agent = AgentSystem(self.rag)

    def answer_question(self, question: str, use_agent=False):
        if use_agent:
            return self.agent.process_query(question)["output"]
        return self.rag.answer_question(question)[0]

    def provide_feedback(self, query, answer, feedback_score):
        docs = self.rag.retriever.invoke(query)
        self.rl.record_feedback(query, docs, answer, feedback_score)


# Run System

In [ ]:
system = ComprehensiveRAGSystem(
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    llm_model="gpt-4o",
    milvus_host="localhost",
    milvus_port="19530",
    collection_name="medical_knowledge_base_v2"
)

query = "What are the symptoms of pneumonia?"
rag_answer = system.answer_question(query)
print("\n🧠 RAG Answer:", rag_answer)

agent_answer = system.answer_question(query, use_agent=True)
print("\n🤖 Agent Answer:", agent_answer)


2025-05-05 14:20:58,303 - INFO - Use pytorch device_name: mps
2025-05-05 14:20:58,303 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-05-05 14:21:03,376 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🧠 RAG Answer: Pneumonia is an infection that inflames the air sacs in one or both lungs, which can fill with fluid or pus. Common symptoms of pneumonia include:

1. Cough, which may produce phlegm (mucus)
2. Fever, which may be mild or high
3. Chills
4. Shortness of breath
5. Chest pain when you breathe or cough
6. Fatigue or tiredness
7. Loss of appetite
8. Nausea or vomiting
9. Confusion, especially in older adults
10. Sweating and shaking chills

It's important to note that symptoms can vary depending on the type of pneumonia, the age and health of the person, and the cause of the infection. If you suspect you or someone else has pneumonia, it is important to seek medical attention for proper diagnosis and treatment.


> Entering new AgentExecutor chain...


2025-05-05 14:21:04,135 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Question: What are the symptoms of pneumonia?
Thought: I need to think about how to solve this. First, I should retrieve relevant documents about pneumonia symptoms.
Action: SearchDocs
Action Input: "symptoms of pneumonia"

2025-05-05 14:21:05,297 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I have retrieved the necessary documents about the symptoms of pneumonia. Now, I should synthesize the information to provide a comprehensive answer.
Action: SynthesizeAnswer
Action Input: "symptoms of pneumonia from retrieved documents"

2025-05-05 14:21:10,030 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Pneumonia is an infection that inflames the air sacs in one or both lungs, which can fill with fluid or pus. The symptoms of pneumonia can vary from mild to severe and may include:

1. Cough, which may produce phlegm (mucus)
2. Fever, sweating, and chills
3. Shortness of breath
4. Rapid, shallow breathing
5. Sharp or stabbing chest pain that worsens when breathing deeply or coughing
6. Loss of appetite, low energy, and fatigue
7. Nausea and vomiting, especially in young children
8. Confusion, especially in older adults
9. Headache
10. Muscle pain

It's important to seek medical attention if you suspect you or someone else has pneumonia, especially if symptoms are severe or if there are risk factors such as age or underlying health conditions.

2025-05-05 14:21:10,468 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I now know the final answer. 

Final Answer: Pneumonia is an infection that inflames the air sacs in one or both lungs, which can fill with fluid or pus. The symptoms of pneumonia can vary from mild to severe and may include cough (which may produce phlegm), fever, sweating, chills, shortness of breath, rapid shallow breathing, sharp or stabbing chest pain that worsens when breathing deeply or coughing, loss of appetite, low energy, fatigue, nausea and vomiting (especially in young children), confusion (especially in older adults), headache, and muscle pain. It's important to seek medical attention if you suspect pneumonia, especially if symptoms are severe or if there are risk factors such as age or underlying health conditions.

> Finished chain.

🤖 Agent Answer: Pneumonia is an infection that inflames the air sacs in one or both lungs, which can fill with fluid or pus. The symptoms of pneumonia can vary from mild to severe and may include cough (which may produce phlegm), fever, swe

In [ ]:
# Activate RL?
#system.provide_feedback(query, answer, feedback_score=0.9)